In [49]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)

## Вычисления коэфициентов

In [50]:
n = 10 
h = 1/n

def u(x): 
    return x*(1-x)
def g(x):
    return x+1
def f(x):
    return 1+5*x-x**3
def p(x):
    return 1+x

fi = np.zeros(n-1)
for i in range(n-1):
    fi[i] = f((i+1)*h)*h**2
print(fi)

[0.015  0.0199 0.0247 0.0294 0.0338 0.0378 0.0416 0.0449 0.0477]


In [51]:
S = np.zeros((n-1)**2)
S = S.reshape(n-1,n-1)

S[0,0] = p(h)+p(2*h) + g(h)*(h**2)
S[0,1] = (-1)*p(2*h)

for i in range(1, n-2):
    S[i,i-1] = (-1)*p((i+1)*h)
    S[i,i] = p((i+1)*h) + p((i+2)*h) + g((i+1)*h)*(h**2)
    S[i,i+1] = (-1)*p((i+2)*h)
    
S[n-2, n-2] = p((n-1)*h) + p(1) + g((n-1)*h)*(h**2)
S[n-2, n-3] = (-1)*p((n-1)*h)

print(S)

[[ 2.311 -1.2    0.     0.     0.     0.     0.     0.     0.   ]
 [-1.2    2.512 -1.3    0.     0.     0.     0.     0.     0.   ]
 [ 0.    -1.3    2.713 -1.4    0.     0.     0.     0.     0.   ]
 [ 0.     0.    -1.4    2.914 -1.5    0.     0.     0.     0.   ]
 [ 0.     0.     0.    -1.5    3.115 -1.6    0.     0.     0.   ]
 [ 0.     0.     0.     0.    -1.6    3.316 -1.7    0.     0.   ]
 [ 0.     0.     0.     0.     0.    -1.7    3.517 -1.8    0.   ]
 [ 0.     0.     0.     0.     0.     0.    -1.8    3.718 -1.9  ]
 [ 0.     0.     0.     0.     0.     0.     0.    -1.9    3.919]]


## Метод прогонки

In [52]:
a = (-1)*S.flatten()[n-1::n]
c = S.flatten()[0::n]
b = (-1)*S.flatten()[1::n]

alpha = np.zeros(n-2)
beta = np.zeros(n-1)
y = np.zeros(n-1)

alpha[0] = b[0]/c[0]
beta[0] = fi[0]/c[0]
for i in range(1, n-2):
    alpha[i] = b[i]/(c[i]-a[i-1]*alpha[i-1])
    beta[i] = (fi[i] + a[i-1]*beta[i-1])/(c[i]-a[i-1]*alpha[i-1])
beta[n-2] = (fi[n-2] + a[n-3]*beta[n-3])/(c[n-2]-a[n-3]*alpha[n-3])
y[n-2] = beta[n-2]
for i in range(n-2,0,-1):
    y[i-1] = alpha[i-1]*y[i]+beta[i-1]
    
print("Tridiagonal: ", y)
print("Embeded: ", np.linalg.solve(S, fi))

Tridiagonal:  [0.0867 0.1545 0.2031 0.2325 0.2426 0.2332 0.2042 0.1558 0.0877]
Embeded:  [0.0867 0.1545 0.2031 0.2325 0.2426 0.2332 0.2042 0.1558 0.0877]


In [53]:
df1 = pd.DataFrame(np.linalg.solve(S, fi))

for i in range(1, n):
    df1.loc[i-1,1] = y[i-1]
    df1.loc[i-1,2] = u(i*h)
    df1.loc[i-1,3] = abs(df1.loc[i-1,2] - df1.loc[i-1,1])
df1 = df1.rename(columns={0: "Embeded",1: "Tridiagonal Y", 2:"u(ih)", 3: "|Y - U(ih)|"})
df1.index += 1 
df1

,Embeded,Tridiagonal Y,u(ih),|Y - U(ih)|
1,0.086689,0.086689,0.09,0.003311
2,0.154458,0.154458,0.16,0.005542
3,0.203116,0.203116,0.21,0.006884
4,0.232520,0.232520,0.24,0.007480
5,0.242561,0.242561,0.25,0.007439
6,0.233154,0.233154,0.24,0.006846
7,0.204236,0.204236,0.21,0.005764
8,0.155760,0.155760,0.16,0.004240
9,0.087689,0.087689,0.09,0.002311


## Итерационные методы

In [54]:
eps = h**3 # Слишком много итераций
def check(x):
    m = (abs(S.dot(x)-fi)).max()
    if m > eps:
        return True
    else:
        return False
    
def jacobi(A, b):
    itr = 0
    x = np.zeros(n-1)
    D = np.diag(A)
    R = A - np.diagflat(D)

    while(check(x)):
        itr +=1
        x = (b - np.dot(R,x)) / D

    return x, itr

In [55]:
y, iter_n = jacobi(S, fi)
print("Last iteration: ", iter_n)
print("Yacobi: ",  y)

Last iteration:  69
Yacobi:  [0.0846 0.1506 0.1981 0.2268 0.2367 0.2278 0.1998 0.1526 0.0861]


In [56]:
df2 = df1.drop(columns = {'u(ih)', '|Y - U(ih)|'})

for i in range(1, n):
    df2.loc[i, 2] = y[i-1]
    df2.loc[i, 3] = abs(df2.iloc[i-1, 1] - y[i-1])
    
df2 = df2.rename(columns={2: "Yacobi Y",3: "|Yacob - Triad|"})
df2

,Embeded,Tridiagonal Y,Yacobi Y,|Yacob - Triad|
1,0.086689,0.086689,0.084596,0.002094
2,0.154458,0.154458,0.150635,0.003823
3,0.203116,0.203116,0.198061,0.005055
4,0.232520,0.232520,0.226783,0.005737
5,0.242561,0.242561,0.236736,0.005824
6,0.233154,0.233154,0.227782,0.005372
7,0.204236,0.204236,0.199806,0.004430
8,0.155760,0.155760,0.152627,0.003133
9,0.087689,0.087689,0.086087,0.001602


In [57]:


def relax(A, b, w):
    itr = 0
    x = np.zeros(n-1)
    x_prev = np.zeros(n-1)
    U = np.diagflat(np.diag(A, 1), 1)
    L = np.diagflat(np.diag(A, -1), -1)
    D = np.diag(A)
    
    while(check(x)):
        itr +=1
        x = (1-w)*x_prev + w*((b - np.dot(L,x) - np.dot(U,x_prev)) / D)
        x_prev = x.copy()
        
    return x, itr

In [58]:
df3 = pd.DataFrame()
for i in range(1,1000):
    v, itr = relax(S, fi, i*0.001)
    df3.loc[i,0] = i*0.001
    df3.loc[i,1] = itr
df3.rename(df3.loc[:, 0], axis='index',  inplace = True)  
df3 = df3.drop(columns = [0])
df3 = df3.rename(columns={1: "Iteration"})
df3.iloc[99::100]

In [71]:
eps = 0.00001
v, itr = relax(S, fi, 0.99)

df4 = df1.drop(columns = {'u(ih)', '|Y - U(ih)|','u(ih)', '|Y - U(ih)|'})

for i in range(1, n):
    df4.loc[i, 2] = v[i-1]
    df4.loc[i, 3] = abs(df4.iloc[i-1, 1] - v[i-1])
df4 = df4.rename(columns={'Tridiagonal':'Y',2: "Relax Y",3: "|Relax - Triad|"})
print(itr)
df4

155


,Embeded,Tridiagonal Y,Relax Y,|Relax - Triad|
1,0.086689,0.086689,0.086668,0.000021
2,0.154458,0.154458,0.154419,0.000039
3,0.203116,0.203116,0.203065,0.000051
4,0.232520,0.232520,0.232462,0.000058
5,0.242561,0.242561,0.242502,0.000059
6,0.233154,0.233154,0.233100,0.000054
7,0.204236,0.204236,0.204192,0.000045
8,0.155760,0.155760,0.155728,0.000032
9,0.087689,0.087689,0.087673,0.000016


In [60]:
df4 = df1.drop(columns = {'u(ih)', '|Y - U(ih)|'})

for i in range(1, n):
    df4.loc[i, 2] = v[i-1]
    df4.loc[i, 3] = abs(df4.iloc[i-1, 1] - v[i-1])
    
df4 = df4.rename(columns={'Tridiagonal':'Y',2: "Relax Y",3: "|Relax - Triad|"})
df4

,Embeded,Relax,Relax Y,|Relax - Triad|
1,0.086689,0.086689,0.084587,0.002102
2,0.154458,0.154458,0.150620,0.003837
3,0.203116,0.203116,0.198041,0.005075
4,0.232520,0.232520,0.226761,0.005759
5,0.242561,0.242561,0.236713,0.005847
6,0.233154,0.233154,0.227762,0.005392
7,0.204236,0.204236,0.199789,0.004447
8,0.155760,0.155760,0.152615,0.003145
9,0.087689,0.087689,0.086081,0.001608
